In [70]:
%pip install torch


   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.2/198.6 MB 6.9 MB/s eta 0:00:29
   ---------------------------------------- 1.0/198.6 MB 12.6 MB/s eta 0:00:16
   ---------------------------------------- 2.0/198.6 MB 15.7 MB/s eta 0:00:13
    --------------------------------------- 2.7/198.6 MB 14.3 MB/s eta 0:00:14
    --------------------------------------- 3.8/198.6 MB 16.4 MB/s eta 0:00:12
    --------------------------------------- 4.7/198.6 MB 16.7 MB/s eta 0:00:12
   - -------------------------------------- 5.8/198.6 MB 16.8 MB/s eta 0:00:12
   - -------------------------------------- 6.7/198.6 MB 17.2 MB/s eta 0:00:12
   - -------------------------------------- 7.9/198.6 MB 18.8 MB/s eta 0:00:11
   - -------------------------------------- 9.0/198.6 MB 18.5 MB/s eta 0:00:11
   -- ------------------------------------- 10.4/198.6 MB 20.5 MB/s eta 0:00:10
   -- ------------------------------------- 11.6/198.6 MB 21

In [8]:

#import biopython

In [38]:
from Bio.PDB import PDBList, PDBParser, MMCIFParser


pdb_id = '1A32'
PDBList().retrieve_pdb_file(pdb_id)

pdb_file = PDBList().retrieve_pdb_file(pdb_id)

structure = MMCIFParser().get_structure(pdb_id, pdb_file)


Structure exists: 'c:\Users\chtim\Documents\EigeneProjekte\Programmieren\Python\LocalAminoacids\a3\1a32.cif' 


In [55]:
from Bio.PDB import NeighborSearch
model = structure[0]
chains = model.get_chains()
chain = next(iter(chains))
residues = chain.get_residues()
print(next(iter(residues)))

#remove all heteroatoms from chain
heteroatoms = []
for residue in chain:
    if(residue.id[0] != ' '):
        #remove residue from chain
        heteroatoms.append(residue.id)

for heteroatom in heteroatoms:
    chain.detach_child(heteroatom)

names = []
#print all residues
for residue in chain:
    names.append(residue.get_resname())

len(names)



<Residue LEU het=  resseq=2 icode= >


85

In [72]:
import torch
aa_alphabet = 'ACDEFGHIKLMNPQRSTVWY' # amino acid alphabet
aa_to_int = {aa: i for i, aa in enumerate(aa_alphabet)} # mapping from amino acid to number

def one_hot_encode(sequence):
    # initialize a zero matrix of shape (len(sequence), len(amino_acids))
    one_hot = torch.zeros(len(sequence), len(aa_alphabet))
    if sequence != "0":
        for i, aa in enumerate(sequence):
            # set the column corresponding to the amino acid to 1
            one_hot[i].scatter_(0, torch.tensor([aa_to_int[aa]]), 1)
    return one_hot

In [91]:
import numpy as np
from Bio.Data.IUPACData import protein_letters_3to1
#distance matix
# Create a list of all atoms in the structure
atoms = [atom for atom in structure.get_atoms()]

# Create a NeighborSearch object
ns = NeighborSearch(atoms)

# Create an empty distance matrix
distance_matrix = []

nClosestAAs = 10
maxRadius = 10.0

aaDistances = np.zeros((len(chain), nClosestAAs, len(aa_alphabet)+1))
# Iterate over each residue in the chain
for index, residue in enumerate(chain):
    # Calculate the distance between the residues
    neighbors = ns.search(residue["CA"].coord, level="R", radius=maxRadius)
    distances = [(protein_letters_3to1[aa.get_resname()[0]+aa.get_resname()[1:].lower()], residue["CA"]-aa["CA"]) for aa in neighbors]
    #sort by distance
    distances.sort(key=lambda x: x[1])
    # Ensure distance is always nClosestAAs large, if too short, fill with 0
    if len(distances) < nClosestAAs:
        distances.extend([("0", 0)] * (nClosestAAs - len(distances)))
    else:
        distances = distances[:nClosestAAs]

    # Print the distances
    aaDistances[index] = [np.append(one_hot_encode(aa[0]),aa[1]) for aa in distances]
    
    #aaDistances[residue.id[1]-1] = distances

print(distances)
print(aaDistances.shape)


[('L', 0.0), ('G', 3.7983356), ('V', 5.2198067), ('L', 5.580389), ('E', 6.4865713), ('K', 7.912012), ('I', 8.589743), ('T', 8.869416), ('R', 9.62728), ('L', 10.502773)]
(85, 10, 21)


In [93]:
torch.from_numpy(aaDistances)

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  3.8177],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  6.2736],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, 10.3375],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  3.8177],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  3.8201],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  9.7444],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, 10.5362],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0